1. Napisz swój program reprezentujący następujący CSP, który ma zbiór zmiennych {X1, X2, X3}; na obrazku pokazana jest domena każdej zmiennej i
    wszystkie ograniczenia między zmiennymi.

In [10]:
# Definicja zmiennych i ich domen
variables = {
    'X1': ['R', 'B', 'G'],
    'X2': ['R'],
    'X3': ['G']
}

# Ograniczenia: zmienne muszą mieć różne wartości
def is_consistent(assignment, var, value):
    for other_var, other_value in assignment.items():
        if other_value == value:
            return False
    return True


2. Załóżmy, że przy przypisywaniu wartości do zmiennych jako pierwszy wybierany jest X2. Napisz program w taki sposób, aby przy wyborze kolejnej zmiennej przestrzegał zasady minimalnych pozostałych wartości (Zobacz MRV
    z wykładu 5).

In [16]:
# Definicja zmiennych i ich domen
variables = {
    'X1': ['R', 'B', 'G'],
    'X2': ['R'],
    'X3': ['G']
}

# Ograniczenie: różne wartości
def is_consistent(assignment, var, value):
    for other_var, other_value in assignment.items():
        if other_value == value:
            return False
    return True

# MRV - wybór zmiennej z najmniejszą dostępną liczbą wartości
def select_unassigned_variable(variables, assignment):
    unassigned = {var: domain for var, domain in variables.items() if var not in assignment}
    return min(unassigned, key=lambda var: len(unassigned[var]))

# Rozwiązanie CSP z użyciem MRV
def backtrack(assignment):
    if len(assignment) == len(variables):
        return assignment

    var = select_unassigned_variable(variables, assignment)

    for value in variables[var]:
        if is_consistent(assignment, var, value):
            assignment[var] = value
            result = backtrack(assignment)
            if result:
                return result
            del assignment[var]

    return None

# Uruchomienie algorytmu
solution = backtrack({})
print(solution)


{'X2': 'R', 'X3': 'G', 'X1': 'B'}


3. Napisz program, aby znaleźć rozwiązanie CSP.

In [12]:
from constraint import Problem, AllDifferentConstraint

problem = Problem()

problem.addVariable("X1", ["R", "B", "G"])
problem.addVariable("X2", ["R"])
problem.addVariable("X3", ["G"])

problem.addConstraint(lambda x1, x2: x1 != x2, ("X1", "X2"))
problem.addConstraint(lambda x1, x3: x1 != x3, ("X1", "X3"))
problem.addConstraint(lambda x2, x3: x2 != x3, ("X2", "X3"))

solutions = problem.getSolutions()

for s in solutions:
    print(s)

{'X2': 'R', 'X3': 'G', 'X1': 'B'}


(4) można rozważyć dowolny z systemów decyzyjnych z pierwszego zestawu
ćwiczeń. Można także rozważyć następujący przykład systemu decyzyjnego.

Napisz program, który znajdzie regułę z danego systemu decyzyjnego zgod-
nie z następującą metodą pokrywania sekwencyjnego.


In [22]:
import pandas as pd
from copy import deepcopy

# Dane wejściowe
data_dict = {
    'o1': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o2': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o3': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o4': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie-pewne'},
    'o5': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie'},
    'o6': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o7': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o8': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o9': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'tak'},
}
df = pd.DataFrame.from_dict(data_dict, orient='index')

target_class = 'tak'
attributes = ['a1', 'a2', 'a3']
positive_examples = df[df['dec'] == target_class]
remaining = set(positive_examples.index)

def get_covered(df, rule):
    covered = df
    for attr, val in rule.items():
        covered = covered[covered[attr] == val]
    return set(covered.index)

rules = []

while remaining:
    current_rule = {}
    available_attrs = [a for a in attributes if a not in current_rule]
    best_precision = 0

    while True:
        candidates = []
        for attr in available_attrs:
            for val in df[attr].unique():
                rule_candidate = deepcopy(current_rule)
                rule_candidate[attr] = val
                covered = get_covered(df, rule_candidate)
                if not covered:
                    continue
                positives = [o for o in covered if df.loc[o, 'dec'] == target_class]
                precision = len(positives) / len(covered)
                candidates.append((precision, len(positives), rule_candidate))

        if not candidates:
            break

        # Wybierz najlepszą regułę
        candidates.sort(key=lambda x: (x[0], x[1]), reverse=True)
        precision, positives_count, best_rule = candidates[0]

        if positives_count == 0:
            break

        if len(best_rule) == len(current_rule):  # nie dodał nic nowego
            break

        current_rule = best_rule
        available_attrs = [a for a in attributes if a not in current_rule]

        # if precision == 1.0:
        #     break

    # Sprawdź, czy pokryliśmy jakieś nowe pozytywne przypadki
    covered_by_rule = get_covered(df, current_rule)
    new_covered = covered_by_rule & remaining
    if not new_covered:
        break  # unikamy nieskończonej pętli

    rules.append((current_rule, target_class))
    remaining -= new_covered

# Wyświetlenie reguł
for idx, (cond, cls) in enumerate(rules):
    cond_str = " ∧ ".join([f"{k} = {v}" for k, v in cond.items()])
    print(f"R{idx+1}: IF {cond_str} THEN dec = {cls}")


R1: IF a1 = wysoka ∧ a2 = bliski ∧ a3 = średni THEN dec = tak


(5) Rozważmy przykład systemu decyzyjnego omawianego na wykładzie (SI-
W6). Załóżmy, że chcemy opisać klasy decyzyjne odpowiadające ‘tak’ i ‘nie’

w odniesieniu do zbioru atrybutów A = {a1, a2, a3}. Oznacza to, że podzbiory
obiektów, które chcemy opisać, to odpowiednio X1 = {o1, o2, o3, o7, o9} i X2
= {o5, o6, o8}. Znajdź opis (tj. przybliżenie dolne i przybliżenie górne) dla
X2 w odniesieniu do A. (ii) Znajdź podobnie opis dla X1 i X2 w odniesieniu
do B = {a1, a2}.

In [14]:
from collections import defaultdict

# Dane systemu decyzyjnego
data = {
    'o1': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o2': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o3': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o4': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie-pewne'},
    'o5': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie'},
    'o6': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o7': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o8': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o9': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'tak'},
}

# Zbiory obiektów
X1 = {'o1', 'o2', 'o3', 'o7', 'o9'} # dec = tak
X2 = {'o4'}                         # dec = nie-pewne
X3 = {'o5', 'o6', 'o8'}             # dec = nie

def rough_set_approximation(data, target_objects, attributes):
    """Zwraca przybliżenie dolne i górne zbioru target_objects względem podanych atrybutów."""
    # Grupowanie obiektów w bloki nieodróżnialności
    blocks = defaultdict(set)
    for obj_id, attr in data.items():
        key = tuple(attr[a] for a in attributes)
        blocks[key].add(obj_id)

    lower_approx = set()
    upper_approx = set()

    for block in blocks.values():
        if block.issubset(target_objects):
            lower_approx.update(block)
        if block & target_objects:
            upper_approx.update(block)

    return lower_approx, upper_approx

def show_result(name, lower, upper):
    print(f"\n>>> {name}")
    print("Przybliżenie dolne:", sorted(lower))
    print("Przybliżenie górne:", sorted(upper))

# Analiza dla A = {a1, a2, a3}
attributes_A = ['a1', 'a2', 'a3']
l_X2_A, u_X2_A = rough_set_approximation(data, X2, attributes_A)
l_X1_A, u_X1_A = rough_set_approximation(data, X1, attributes_A)

# Analiza dla B = {a1, a2}
attributes_B = ['a1', 'a2']
l_X2_B, u_X2_B = rough_set_approximation(data, X2, attributes_B)
l_X1_B, u_X1_B = rough_set_approximation(data, X1, attributes_B)

# Wyniki
show_result("X2 w odniesieniu do A = {a1, a2, a3}", l_X2_A, u_X2_A)
show_result("X1 w odniesieniu do A = {a1, a2, a3}", l_X1_A, u_X1_A)
show_result("X2 w odniesieniu do B = {a1, a2}", l_X2_B, u_X2_B)
show_result("X1 w odniesieniu do B = {a1, a2}", l_X1_B, u_X1_B)



>>> X2 w odniesieniu do A = {a1, a2, a3}
Przybliżenie dolne: []
Przybliżenie górne: ['o4', 'o5']

>>> X1 w odniesieniu do A = {a1, a2, a3}
Przybliżenie dolne: ['o1', 'o2', 'o3', 'o7']
Przybliżenie górne: ['o1', 'o2', 'o3', 'o6', 'o7', 'o8', 'o9']

>>> X2 w odniesieniu do B = {a1, a2}
Przybliżenie dolne: []
Przybliżenie górne: ['o4', 'o5', 'o6', 'o8', 'o9']

>>> X1 w odniesieniu do B = {a1, a2}
Przybliżenie dolne: ['o1', 'o2', 'o3', 'o7']
Przybliżenie górne: ['o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9']


(4) można rozważyć dowolny z systemów decyzyjnych z pierwszego zestawu
ćwiczeń. Można także rozważyć następujący przykład systemu decyzyjnego.

Napisz program, który znajdzie regułę z danego systemu decyzyjnego zgod-
nie z następującą metodą pokrywania sekwencyjnego.

In [33]:
import pandas as pd
from copy import deepcopy

# Dane wejściowe
# data_dict = {
#     'o1': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
#     'o2': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
#     'o3': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
#     'o4': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie-pewne'},
#     'o5': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie'},
#     'o6': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
#     'o7': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
#     'o8': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
#     'o9': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'tak'},
# }

data_dict = {
    'o1': {'a1':1, 'a2':1, 'a3':1, 'a4':1, 'a5':1, 'a6':1, 'a7':1, 'dec':1},
    'o2': {'a1':1, 'a2':1, 'a3':1, 'a4':1, 'a5':1, 'a6':1, 'a7':1, 'dec':1},
    'o3': {'a1':1, 'a2':1, 'a3':1, 'a4':1, 'a5':2, 'a6':2, 'a7':2, 'dec':2},
    'o4': {'a1':1, 'a2':1, 'a3':3, 'a4':3, 'a5':1, 'a6':1, 'a7':2, 'dec':2},
    'o5': {'a1':3, 'a2':3, 'a3':2, 'a4':3, 'a5':2, 'a6':2, 'a7':3, 'dec':4},
    'o6': {'a1':1, 'a2':2, 'a3':1, 'a4':2, 'a5':1, 'a6':2, 'a7':1, 'dec':1},
    'o7': {'a1':1, 'a2':1, 'a3':0, 'a4':1, 'a5':0, 'a6':1, 'a7':0, 'dec':1}
}


df = pd.DataFrame.from_dict(data_dict, orient='index')

target_class = 1
attributes = ['a1', 'a2', 'a3']

positive_examples = df[df['dec'] == target_class]
remaining = set(positive_examples.index)

def get_covered(df, rule):
    covered = df
    for attr, val in rule.items():
        covered = covered[covered[attr] == val]
    return set(covered.index)

rules = []

while remaining:
    current_rule = {}
    available_attrs = [a for a in attributes if a not in current_rule]
    best_precision = 0

    while True:
        candidates = []
        for attr in available_attrs:
            for val in df[attr].unique():
                rule_candidate = deepcopy(current_rule)
                rule_candidate[attr] = val
                covered = get_covered(df, rule_candidate)
                if not covered:
                    continue
                positives = [o for o in covered if df.loc[o, 'dec'] == target_class]
                precision = len(positives) / len(covered)
                candidates.append((precision, len(positives), rule_candidate))

        if not candidates:
            break

        # Wybierz najlepszą regułę
        candidates.sort(key=lambda x: (x[0], x[1]), reverse=True)
        precision, positives_count, best_rule = candidates[0]

        if positives_count == 0:
            break

        if len(best_rule) == len(current_rule):  # nie dodał nic nowego
            break

        current_rule = best_rule
        available_attrs = [a for a in attributes if a not in current_rule]

        # if precision == 1.0:
        #     break

    # Sprawdź, czy pokryliśmy jakieś nowe pozytywne przypadki
    covered_by_rule = get_covered(df, current_rule)
    new_covered = covered_by_rule & remaining
    if not new_covered:
        break  # unikamy nieskończonej pętli

    rules.append((current_rule, target_class))
    remaining -= new_covered

# Wyświetlenie reguł
for idx, (cond, cls) in enumerate(rules):
    cond_str = " ∧ ".join([f"{k} = {v}" for k, v in cond.items()])
    print(f"R{idx+1}: IF {cond_str} THEN dec = {cls}")


R1: IF a2 = 2 ∧ a1 = 1 ∧ a3 = 1 THEN dec = 1


(5) Rozważmy przykład systemu decyzyjnego omawianego na wykładzie (SI-
W6). Załóżmy, że chcemy opisać klasy decyzyjne odpowiadające ‘tak’ i ‘nie’

w odniesieniu do zbioru atrybutów A = {a1, a2, a3}. Oznacza to, że podzbiory
obiektów, które chcemy opisać, to odpowiednio X1 = {o1, o2, o3, o7, o9} i X2
= {o5, o6, o8}. Znajdź opis (tj. przybliżenie dolne i przybliżenie górne) dla
X2 w odniesieniu do A. (ii) Znajdź podobnie opis dla X1 i X2 w odniesieniu
do B = {a1, a2}.

In [7]:
from collections import defaultdict

# Dane systemu decyzyjnego – każdy obiekt ma wartości atrybutów oraz decyzję końcową
data = {
    'o1': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o2': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o3': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o4': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie'},
    'o5': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie'},
    'o6': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o7': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o8': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o9': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'tak'},
}

# Zbiory obiektów dla klas decyzyjnych
X1 = {'o1', 'o2', 'o3', 'o7', 'o9'}  # Klasa decyzyjna: tak
X2 = {'o5', 'o6', 'o8'}             # Klasa decyzyjna: nie

def rough_set_approximation(data, target_objects, attributes):
    """
    Oblicza przybliżenie dolne i górne zbioru target_objects względem atrybutów.
    Uwzględnia spójność decyzji w blokach nieodróżnialności.
    
    Args:
        data (dict): dane wejściowe z atrybutami i decyzjami
        target_objects (set): obiekty należące do analizowanego zbioru
        attributes (list): lista atrybutów do rozważenia

    Returns:
        (set, set): przybliżenie dolne i przybliżenie górne
    """
    # Grupowanie obiektów w bloki nieodróżnialności
    blocks = defaultdict(set)
    for obj_id, attr in data.items():
        # Tworzymy klucz na podstawie wartości wybranych atrybutów
        key = tuple(attr[a] for a in attributes)
        blocks[key].add(obj_id)

    # Zakładamy, że wszystkie obiekty z target_objects mają tę samą decyzję
    target_decision = data[next(iter(target_objects))]['dec']

    lower_approx = set()
    upper_approx = set()

    for block in blocks.values():
        # Pobierz zbiór decyzji w bloku
        decisions_in_block = {data[obj]['dec'] for obj in block}
        
        # Blok brany pod uwagę tylko, jeśli zawiera wyłącznie jedną decyzję
        if decisions_in_block == {target_decision}:
            # Cały blok zawiera się w zbiorze – należy do przybliżenia dolnego
            if block.issubset(target_objects):
                lower_approx.update(block)
            # Część bloku przecina się ze zbiorem – należy do przybliżenia górnego
            if block & target_objects:
                upper_approx.update(block)

    return lower_approx, upper_approx

def show_result(name, lower, upper):
    """Funkcja pomocnicza do prezentacji wyników"""
    print(f"\n>>> {name}")
    print("Przybliżenie dolne:", sorted(lower))
    print("Przybliżenie górne:", sorted(upper))

# Analiza dla zbioru cech A = {a1, a2, a3}
attributes_A = ['a1', 'a2', 'a3']
l_X2_A, u_X2_A = rough_set_approximation(data, X2, attributes_A)
l_X1_A, u_X1_A = rough_set_approximation(data, X1, attributes_A)

# Analiza dla zbioru cech B = {a1, a2}
attributes_B = ['a1', 'a2']
l_X2_B, u_X2_B = rough_set_approximation(data, X2, attributes_B)
l_X1_B, u_X1_B = rough_set_approximation(data, X1, attributes_B)

# Wyświetlenie wyników
show_result("X2 w odniesieniu do A = {a1, a2, a3}", l_X2_A, u_X2_A)
show_result("X1 w odniesieniu do A = {a1, a2, a3}", l_X1_A, u_X1_A)
show_result("X2 w odniesieniu do B = {a1, a2}", l_X2_B, u_X2_B)
show_result("X1 w odniesieniu do B = {a1, a2}", l_X1_B, u_X1_B)


>>> X2 w odniesieniu do A = {a1, a2, a3}
Przybliżenie dolne: []
Przybliżenie górne: ['o4', 'o5']

>>> X1 w odniesieniu do A = {a1, a2, a3}
Przybliżenie dolne: ['o1', 'o2', 'o3', 'o7']
Przybliżenie górne: ['o1', 'o2', 'o3', 'o7']

>>> X2 w odniesieniu do B = {a1, a2}
Przybliżenie dolne: []
Przybliżenie górne: []

>>> X1 w odniesieniu do B = {a1, a2}
Przybliżenie dolne: ['o1', 'o2', 'o3', 'o7']
Przybliżenie górne: ['o1', 'o2', 'o3', 'o7']


(6) Szukamy w obiektach systemu decyzyjnego, począwszy od pierwszego, a skończy-
wszy na ostatnim reguł długości jeden, które są niesprzeczne. Po znalezieniu

reguły niesprzecznej, dany obiekt wyrzucamy z rozważań, pamiętając o tym, że
dalej bierze udział w sprawdzaniu sprzeczności i może wspierać inne reguły.
Jeśli po przeszukaniu wszystkich obiektów, pozostają obiekty nie wyrzucone

z rozważań, szukamy w nich kombinacji niesprzecznej długości dwa i postępu-
jemy analogicznie jak w przypadku reguł pierwszego rzędu. Wyszukiwanie reguł

niesprzecznych jest kontynuowane do momentu wyeliminowania wszystkich obiek-
tów niesprzecznych. Jeśli w systemie pojawią się obiekty, które są sprzeczne na

wszystkich deskryptorach, nie kreujemy z nich reguł.

In [1]:
import pandas as pd
from itertools import combinations

data = {
    'ob': ['o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8'],
    'a1': [1, 1, 1, 1, 1, 1, 1, 1],
    'a2': [1, 1, 1, 1, 1, 1, 1, 1],
    'a3': [1, 1, 1, 1, 2, 2, 2, 2],
    'a4': [1, 1, 3, 3, 1, 1, 2, 2],
    'a5': [3, 3, 2, 3, 2, 2, 3, 4],
    'a6': [1, 2, 1, 2, 1, 2, 1, 1],
    'd':  [1, 1, 0, 1, 0, 1, 0, 1]
}
df = pd.DataFrame(data)

def sequential_covering_verbose(df):
    rules_by_length = {}
    remaining_df = df.copy()
    attributes = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6']

    for rule_len in range(1, len(attributes) + 1):
        marked_for_removal = []

        for idx, row in remaining_df.iterrows():
            matched = False

            for attr_comb in combinations(attributes, rule_len):
                condition = (remaining_df[list(attr_comb)] == row[list(attr_comb)]).all(axis=1)
                covered = remaining_df[condition]

                if all(covered['d'] == row['d']):
                    rule = {
                        'object': row['ob'],
                        'attrs': attr_comb,
                        'values': row[list(attr_comb)].to_dict(),
                        'decision': row['d'],
                        'covered_objects': covered['ob'].tolist()
                    }
                    rules_by_length.setdefault(rule_len, []).append(rule)
                    marked_for_removal.extend(covered.index.tolist())
                    matched = True
                    break 

            if not matched:
                rules_by_length.setdefault(rule_len, []).append({
                    'object': row['ob'],
                    'no_rule': True
                })

        remaining_df = remaining_df.drop(index=set(marked_for_removal))
        if remaining_df.empty:
            break

    return rules_by_length


rules_by_length = sequential_covering_verbose(df)

# Wyświetlanie wyników
print()
for length in sorted(rules_by_length.keys()):
    print(f"Reguły: rząd-{length}:")
    for rule in rules_by_length[length]:
        if 'no_rule' in rule:
            print(f"{rule['object']}: brak reguły")
        else:
            conds = ' ∧ '.join([f"{attr} = {value}" for attr, value in rule['values'].items()])
            covered = ", ".join(rule['covered_objects'])
            print(f"{rule['object']}: ({conds}) ⇒ (d = {rule['decision']}) "
                  f"[{len(rule['covered_objects'])}], wyrzucamy z rozważań obiekt{'y' if len(rule['covered_objects']) > 1 else ''} {covered}.")


Reguły: rząd-1:
o1: brak reguły
o2: (a6 = 2) ⇒ (d = 1) [3], wyrzucamy z rozważań obiekty o2, o4, o6.
o3: brak reguły
o4: (a6 = 2) ⇒ (d = 1) [3], wyrzucamy z rozważań obiekty o2, o4, o6.
o5: brak reguły
o6: (a6 = 2) ⇒ (d = 1) [3], wyrzucamy z rozważań obiekty o2, o4, o6.
o7: brak reguły
o8: (a5 = 4) ⇒ (d = 1) [1], wyrzucamy z rozważań obiekt o8.
Reguły: rząd-2:
o1: (a3 = 1 ∧ a4 = 1) ⇒ (d = 1) [1], wyrzucamy z rozważań obiekt o1.
o3: (a1 = 1 ∧ a4 = 3) ⇒ (d = 0) [1], wyrzucamy z rozważań obiekt o3.
o5: (a1 = 1 ∧ a3 = 2) ⇒ (d = 0) [2], wyrzucamy z rozważań obiekty o5, o7.
o7: (a1 = 1 ∧ a3 = 2) ⇒ (d = 0) [2], wyrzucamy z rozważań obiekty o5, o7.


In [ ]:
# Ponowne zaimportowanie biblioteki pandas, ponieważ poprzedni stan został utracony
import pandas as pd

# Dane wejściowe
data_dict = {
    'o1': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o2': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o3': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o4': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie-pewne'},
    'o5': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie'},
    'o6': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o7': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o8': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o9': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'tak'},
}

df = pd.DataFrame.from_dict(data_dict, orient='index')

# Klasy równoważności względem a1, a2, a3
equiv_classes = df.groupby(['a1', 'a2', 'a3']).apply(lambda x: list(x.index)).tolist()

# Generalized decision function ∂A
generalized_decisions = {obj: set(df.loc[df[['a1', 'a2', 'a3']].eq(row[['a1', 'a2', 'a3']]).all(axis=1)].dec)
                         for obj, row in df.iterrows()}

# Rough membership function μA
rough_memberships = {}
decision_labels = df['dec'].unique().tolist()

for obj, row in df.iterrows():
    class_members = df[(df['a1'] == row['a1']) & (df['a2'] == row['a2']) & (df['a3'] == row['a3'])]
    class_size = len(class_members)
    counts = class_members['dec'].value_counts().to_dict()
    print(counts)
    membership = [counts.get(dec, 0)/class_size for dec in decision_labels]
    rough_memberships[obj] = membership

equiv_classes, generalized_decisions, rough_memberships

{'tak': 4}
{'tak': 4}
{'tak': 4}
{'nie-pewne': 1, 'nie': 1}
{'nie-pewne': 1, 'nie': 1}
{'nie': 2, 'tak': 1}
{'tak': 4}
{'nie': 2, 'tak': 1}
{'nie': 2, 'tak': 1}


/tmp/ipykernel_38942/663564701.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  equiv_classes = df.groupby(['a1', 'a2', 'a3']).apply(lambda x: list(x.index)).tolist()


([['o6', 'o8', 'o9'], ['o4', 'o5'], ['o1', 'o2', 'o3', 'o7']],
 {'o1': {'tak'},
  'o2': {'tak'},
  'o3': {'tak'},
  'o4': {'nie', 'nie-pewne'},
  'o5': {'nie', 'nie-pewne'},
  'o6': {'nie', 'tak'},
  'o7': {'tak'},
  'o8': {'nie', 'tak'},
  'o9': {'nie', 'tak'}},
 {'o1': [1.0, 0.0, 0.0],
  'o2': [1.0, 0.0, 0.0],
  'o3': [1.0, 0.0, 0.0],
  'o4': [0.0, 0.5, 0.5],
  'o5': [0.0, 0.5, 0.5],
  'o6': [0.3333333333333333, 0.0, 0.6666666666666666],
  'o7': [1.0, 0.0, 0.0],
  'o8': [0.3333333333333333, 0.0, 0.6666666666666666],
  'o9': [0.3333333333333333, 0.0, 0.6666666666666666]})

In [17]:
from collections import defaultdict

def rough_set_approximations(data, keys, target_decision):
    # Grupy ekwiwalencji: klucz = wartości atrybutów z 'keys', wartość = lista obiektów
    equivalence_classes = defaultdict(list)
    
    for obj_id, attributes in data.items():
        key = tuple(attributes[k] for k in keys)
        equivalence_classes[key].append((obj_id, attributes['dec']))
    
    lower_approx = set()
    upper_approx = set()
    
    for group in equivalence_classes.values():
        decisions = [dec for _, dec in group]
        obj_ids = [obj_id for obj_id, _ in group]
        
        if all(dec == target_decision for dec in decisions):
            lower_approx.update(obj_ids)
        
        if any(dec == target_decision for dec in decisions):
            upper_approx.update(obj_ids)
    
    return lower_approx, upper_approx


In [24]:
data = {
    'o1': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o2': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o3': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o4': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie-pewne'},
    'o5': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'silny', 'dec': 'nie'},
    'o6': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o7': {'a1': 'wysoka', 'a2': 'bliski', 'a3': 'średni', 'dec': 'tak'},
    'o8': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'nie'},
    'o9': {'a1': 'więcej niż średnia', 'a2': 'daleki', 'a3': 'lekki', 'dec': 'tak'},
}

# Przykład użycia: tylko a1 i a2, szukamy obiektów z decyzją "tak"
lower, upper = rough_set_approximations(data, keys=['a1', 'a2'], target_decision='tak')

print("Ograniczenie dolne:", lower)
print("Ograniczenie górne:", upper)


Ograniczenie dolne: {'o1', 'o2', 'o3', 'o7'}
Ograniczenie górne: {'o1', 'o7', 'o6', 'o8', 'o9', 'o3', 'o5', 'o2', 'o4'}
